# AdventureWorks - Predicting customer churn

*NOTE: Due to featuretools' memory requirement, this notebook will not run on MyBinder.*

In this notebook, we will demonstrate how getML can be used for a customer churn project using a synthetic dataset of a fictional company. We will also benchmark getML against featuretools.

Summary:

- Prediction type: __Classification model__
- Domain: __Customer loyalty__
- Prediction target: __churn__ 
- Population size: __19704__

_Author: Dr. Patrick Urbanke_

# Background

[AdventureWorks](https://github.com/Microsoft/sql-server-samples/releases/tag/adventureworks) is a fictional company that sells bicycles. It is used by Microsoft to showcase how its MS SQL Server can be used to manage business data. Since this dataset resembles a real-world customer database and it is open-source, we will use it to showcase how getML can be used for a classic customer churn project (real customer databases are not easily available for the purposes of showcasing and benchmarking, for reasons of data privacy).

The dataset has been downloaded from the [CTU Prague relational learning repository](https://relational.fit.cvut.cz/dataset/AdventureWorks) (Motl and Schulte, 2015).

We will benchmark [getML](https://www.getml.com) 's feature learning algorithms against [featuretools](https://www.featuretools.com), an open-source implementation of the propositionalization algorithm, similar to getML's FastProp.

### A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

### Where is this running?

Your getML live session is running inside a docker container on [mybinder.org](https://mybinder.org/), a service built by the Jupyter community and funded by Google Cloud, OVH, GESIS Notebooks and the Turing Institute. As it is a free service, this session will shut down after 10 minutes of inactivity.

# Analysis

Let's get started with the analysis and set up your session:

In [1]:
import copy
import os
from urllib import request

import numpy as np
import pandas as pd
from IPython.display import Image
import matplotlib.pyplot as plt
%matplotlib inline  

import featuretools
import getml

getml.engine.launch()
getml.set_project('adventure_works')

/home/ubuntu/.local/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


getML engine is already running.

Connected to project 'adventure_works'


## 1. Loading data

### 1.1 Download from source

We begin by downloading the data:

In [2]:
conn = getml.database.connect_mariadb(
    host="relational.fit.cvut.cz",
    dbname="AdventureWorks2014",
    port=3306,
    user="guest",
    password="relational"
)

conn

Connection(dbname='AdventureWorks2014',
           dialect='mysql',
           host='relational.fit.cvut.cz',
           port=3306)

In [3]:
def load_if_needed(name):
    """
    Loads the data from the relational learning
    repository, if the data frame has not already
    been loaded.
    """
    if not getml.data.exists(name):
        data_frame = getml.DataFrame.from_db(
            name=name,
            table_name=name,
            conn=conn
        )
        data_frame.save()
    else:
        data_frame = getml.data.load_data_frame(name)
    return data_frame

In [4]:
product = load_if_needed("Product")
sales_order_detail = load_if_needed("SalesOrderDetail")
sales_order_header = load_if_needed("SalesOrderHeader")
sales_order_reason = load_if_needed("SalesOrderHeaderSalesReason")
special_offer = load_if_needed("SpecialOffer")
store = load_if_needed("Store")

In [5]:
product

name,ProductID,MakeFlag,ProductSubcategoryID,ProductModelID,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,FinishedGoodsFlag,DaysToManufacture,Name,ProductNumber,Color,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,ProductLine,Class,Style,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
role,join_key,categorical,categorical,categorical,numerical,numerical,numerical,numerical,unused_float,unused_float,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string
0,1,0,nan,nan,1000,750,0,0,0,0,Adjustable Race,AR-5381,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,694215B7-08F7-4C0D-ACB1-D734BA44...,2014-02-08 10:01:36
1,2,0,nan,nan,1000,750,0,0,0,0,Bearing Ball,BA-8327,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,58AE3C20-4F3A-4749-A7D4-D568806C...,2014-02-08 10:01:36
2,3,1,nan,nan,800,600,0,0,0,1,BB Ball Bearing,BE-2349,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,9C21AED2-5BFA-4F18-BCB8-F11638DC...,2014-02-08 10:01:36
3,4,0,nan,nan,800,600,0,0,0,0,Headset Ball Bearings,BE-2908,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,ECFED6CB-51FF-49B5-B06C-7D8AC834...,2014-02-08 10:01:36
4,316,1,nan,nan,800,600,0,0,0,1,Blade,BL-2036,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,E73E9750-603B-4131-89F5-3DD15ED5...,2014-02-08 10:01:36
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,995,1,5,96,500,375,44.9506,101.24,1,1,ML Bottom Bracket,BB-8107,NULL,NULL,NULL,G,168.00,NULL,M,NULL,2013-05-30 00:00:00,NULL,NULL,71AB847F-D091-42D6-B735-7B0C2D82...,2014-02-08 10:01:36
500,996,1,5,97,500,375,53.9416,121.49,1,1,HL Bottom Bracket,BB-9108,NULL,NULL,NULL,G,170.00,NULL,H,NULL,2013-05-30 00:00:00,NULL,NULL,230C47C5-08B2-4CE3-B706-69C0BDD6...,2014-02-08 10:01:36
501,997,1,2,31,100,75,343.6496,539.99,1,4,"Road-750 Black, 44",BK-R19B-44,Black,44,CM,LB,19.77,R,L,U,2013-05-30 00:00:00,NULL,NULL,44CE4802-409F-43AB-9B27-CA534218...,2014-02-08 10:01:36


In [6]:
sales_order_detail

name,ModifiedDate,SalesOrderID,SalesOrderDetailID,ProductID,SpecialOfferID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal,CarrierTrackingNumber,rowguid
role,time_stamp,join_key,join_key,join_key,join_key,numerical,numerical,numerical,numerical,unused_string,unused_string
unit,"time stamp, comparison only",,,,,,,,,,
0,2011-05-31,43659,1,776,1,1,2024.994,0,2024.994,4911-403C-98,B207C96D-D9E6-402B-8470-2CC176C4...
1,2011-05-31,43659,2,777,1,3,2024.994,0,6074.982,4911-403C-98,7ABB600D-1E77-41BE-9FE5-B9142CFC...
2,2011-05-31,43659,3,778,1,1,2024.994,0,2024.994,4911-403C-98,475CF8C6-49F6-486E-B0AD-AFC6A50C...
3,2011-05-31,43659,4,771,1,1,2039.994,0,2039.994,4911-403C-98,04C4DE91-5815-45D6-8670-F462719F...
4,2011-05-31,43659,5,772,1,1,2039.994,0,2039.994,4911-403C-98,5A74C7D2-E641-438E-A7AC-37BF2328...
,...,...,...,...,...,...,...,...,...,...,...
121312,2014-06-30,75122,121313,878,1,1,21.98,0,21.98,NULL,8CAD6675-18CC-4F47-8287-97B41A8E...
121313,2014-06-30,75122,121314,712,1,1,8.99,0,8.99,NULL,84F1C363-1C50-4442-BE16-541C59B6...


In [7]:
sales_order_header

name,OrderDate,DueDate,ShipDate,ModifiedDate,CustomerID,SalesOrderID,SalesPersonID,TerritoryID,RevisionNumber,OnlineOrderFlag,SalesPersonIDCat,TerritoryIDCat,ShipMethodID,SubTotal,TaxAmt,Freight,TotalDue,Status,BillToAddressID,ShipToAddressID,CreditCardID,CurrencyRateID,churn,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CreditCardApprovalCode,Comment,rowguid
role,time_stamp,time_stamp,time_stamp,time_stamp,join_key,join_key,join_key,join_key,categorical,categorical,categorical,categorical,categorical,numerical,numerical,numerical,numerical,unused_float,unused_float,unused_float,unused_float,unused_float,unused_float,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string
unit,"time stamp, comparison only","time stamp, comparison only","time stamp, comparison only","time stamp, comparison only",,,,,,,,,,,,,,,,,,,,,,,,,
0,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29825,43659,279,5,8,0,279,5,5,20565.6206,1971.5149,616.0984,23153.2339,5,985,985,16281,nan,0,SO43659,PO522145787,10-4020-000676,105041Vi84182,NULL,79B65321-39CA-4115-9CBA-8FE0903E...
1,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29672,43660,279,5,8,0,279,5,5,1294.2529,124.2483,38.8276,1457.3288,5,921,921,5618,nan,0,SO43660,PO18850127500,10-4020-000117,115213Vi29411,NULL,738DC42D-D03B-48A1-9822-F95A67EA...
2,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29734,43661,282,6,8,0,282,6,5,32726.4786,3153.7696,985.553,36865.8012,5,517,517,1346,4,0,SO43661,PO18473189620,10-4020-000442,85274Vi6854,NULL,D91B9131-18A4-4A11-BC3A-90B6F53E...
3,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29994,43662,282,6,8,0,282,6,5,28832.5289,2775.1646,867.2389,32474.9324,5,482,482,10456,4,0,SO43662,PO18444174044,10-4020-000227,125295Vi53935,NULL,4A1ECFC0-CC3A-4740-B028-1C50BB48...
4,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29565,43663,276,4,8,0,276,4,5,419.4589,40.2681,12.5838,472.3108,5,1073,1073,4322,nan,0,SO43663,PO18009186470,10-4020-000510,45303Vi22691,NULL,9B1E7A40-6AE0-4AD3-811C-A6495185...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31460,2014-06-30,2014-07-12,2014-07-07,2014-07-07,11981,75119,nan,1,8,1,nan,1,1,42.28,3.3824,1.057,46.7194,5,17649,17649,6761,nan,nan,SO75119,NULL,10-4030-011981,429826Vi35166,NULL,9382F1C9-383A-435F-9449-0EECEA21...
31461,2014-06-30,2014-07-12,2014-07-07,2014-07-07,18749,75120,nan,6,8,1,nan,6,1,84.96,6.7968,2.124,93.8808,5,28374,28374,8925,nan,nan,SO75120,NULL,10-4030-018749,929849Vi46003,NULL,AE6A4FCF-FF73-4CD4-AF2C-5993D00D...


In [8]:
sales_order_reason

name,SalesOrderID,SalesReasonID,ModifiedDate
role,join_key,categorical,unused_string
0,43697,5,2011-05-31 00:00:00
1,43697,9,2011-05-31 00:00:00
2,43702,5,2011-06-01 00:00:00
3,43702,9,2011-06-01 00:00:00
4,43703,5,2011-06-01 00:00:00
,...,...,...
27642,75119,1,2014-06-30 00:00:00
27643,75120,1,2014-06-30 00:00:00
27644,75121,1,2014-06-30 00:00:00


In [9]:
special_offer

name,StartDate,EndDate,SpecialOfferID,Category,Description,Type,MinQty,DiscountPct,MaxQty,rowguid,ModifiedDate
role,time_stamp,time_stamp,join_key,categorical,categorical,categorical,numerical,numerical,unused_float,unused_string,unused_string
unit,"time stamp, comparison only","time stamp, comparison only",,,,,,,,,
0,2011-05-01,2014-11-30,1,No Discount,No Discount,No Discount,0,0,nan,0290C4F5-191F-4337-AB6B-0A2DDE03...,2011-04-01 00:00:00
1,2011-05-31,2014-05-30,2,Reseller,Volume Discount 11 to 14,Volume Discount,11,0.02,14,D7542EE7-15DB-4541-985C-5CC27AEF...,2011-05-01 00:00:00
2,2011-05-31,2014-05-30,3,Reseller,Volume Discount 15 to 24,Volume Discount,15,0.05,24,4BDBCC01-8CF7-40A9-B643-40EC5B71...,2011-05-01 00:00:00
3,2011-05-31,2014-05-30,4,Reseller,Volume Discount 25 to 40,Volume Discount,25,0.1,40,504B5E85-8F3F-4EBC-9E1D-C1BC5DEA...,2011-05-01 00:00:00
4,2011-05-31,2014-05-30,5,Reseller,Volume Discount 41 to 60,Volume Discount,41,0.15,60,677E1D9D-944F-4E81-90E8-47EB0A82...,2011-05-01 00:00:00
,...,...,...,...,...,...,...,...,...,...,...
11,2013-05-30,2013-07-14,12,Reseller,LL Road Frame Sale,Excess Inventory,0,0.35,nan,C0AF1C89-9722-4235-9248-3FBA4D9E...,2013-04-30 00:00:00
12,2013-05-30,2013-08-29,13,Reseller,Touring-3000 Promotion,New Product,0,0.15,nan,5061CCE4-E021-45A8-9A75-DFB36CBB...,2013-04-30 00:00:00


In [10]:
store

name,test,SalesPersonID,BusinessEntityID,Name,Demographics,rowguid,ModifiedDate
role,time_stamp,join_key,unused_float,unused_string,unused_string,unused_string,unused_string
unit,"time stamp, comparison only",,,,,,
0,2014-09-12 11:15:07,279,292,Next-Door Bike Store,"<StoreSurvey xmlns=""http://schem...",A22517E3-848D-4EBE-B9D9-7437F343...,2014-09-12 11:15:07
1,NULL,276,294,Professional Sales and Service,"<StoreSurvey xmlns=""http://schem...",B50CA50B-C601-4A13-B07E-2C63862D...,2014-09-12 11:15:07
2,2014-09-12 11:15:07,277,296,Riders Company,"<StoreSurvey xmlns=""http://schem...",337C3688-1339-4E1A-A08A-B54B2356...,2014-09-12 11:15:07
3,NULL,275,298,The Bike Mechanics,"<StoreSurvey xmlns=""http://schem...",7894F278-F0C8-4D16-BD75-213FDBF1...,2014-09-12 11:15:07
4,NULL,286,300,Nationwide Supply,"<StoreSurvey xmlns=""http://schem...",C3FC9705-A8C4-4F3A-9550-EB2FA4B7...,2014-09-12 11:15:07
,...,...,...,...,...,...,...
696,NULL,282,1988,Retreat Inn,"<StoreSurvey xmlns=""http://schem...",EA21EC81-1BFA-4A07-9B4D-73D9852A...,2014-09-12 11:15:07
697,2014-09-12 11:15:07,281,1990,Technical Parts Manufacturing,"<StoreSurvey xmlns=""http://schem...",C8E3C4ED-8F58-4DB2-B600-E0CD11D9...,2014-09-12 11:15:07


### 1.2 Prepare data for getML

getML requires that we define *roles* for each of the columns.

In [11]:
product.set_role("ProductID", getml.data.roles.join_key)
product.set_role(["MakeFlag", "ProductSubcategoryID", "ProductModelID"], getml.data.roles.categorical)
product.set_role(["SafetyStockLevel", "ReorderPoint", "StandardCost", "ListPrice"], getml.data.roles.numerical)

product

name,ProductID,MakeFlag,ProductSubcategoryID,ProductModelID,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,FinishedGoodsFlag,DaysToManufacture,Name,ProductNumber,Color,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,ProductLine,Class,Style,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
role,join_key,categorical,categorical,categorical,numerical,numerical,numerical,numerical,unused_float,unused_float,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string
0,1,0,nan,nan,1000,750,0,0,0,0,Adjustable Race,AR-5381,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,694215B7-08F7-4C0D-ACB1-D734BA44...,2014-02-08 10:01:36
1,2,0,nan,nan,1000,750,0,0,0,0,Bearing Ball,BA-8327,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,58AE3C20-4F3A-4749-A7D4-D568806C...,2014-02-08 10:01:36
2,3,1,nan,nan,800,600,0,0,0,1,BB Ball Bearing,BE-2349,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,9C21AED2-5BFA-4F18-BCB8-F11638DC...,2014-02-08 10:01:36
3,4,0,nan,nan,800,600,0,0,0,0,Headset Ball Bearings,BE-2908,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,ECFED6CB-51FF-49B5-B06C-7D8AC834...,2014-02-08 10:01:36
4,316,1,nan,nan,800,600,0,0,0,1,Blade,BL-2036,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,E73E9750-603B-4131-89F5-3DD15ED5...,2014-02-08 10:01:36
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,995,1,5,96,500,375,44.9506,101.24,1,1,ML Bottom Bracket,BB-8107,NULL,NULL,NULL,G,168.00,NULL,M,NULL,2013-05-30 00:00:00,NULL,NULL,71AB847F-D091-42D6-B735-7B0C2D82...,2014-02-08 10:01:36
500,996,1,5,97,500,375,53.9416,121.49,1,1,HL Bottom Bracket,BB-9108,NULL,NULL,NULL,G,170.00,NULL,H,NULL,2013-05-30 00:00:00,NULL,NULL,230C47C5-08B2-4CE3-B706-69C0BDD6...,2014-02-08 10:01:36
501,997,1,2,31,100,75,343.6496,539.99,1,4,"Road-750 Black, 44",BK-R19B-44,Black,44,CM,LB,19.77,R,L,U,2013-05-30 00:00:00,NULL,NULL,44CE4802-409F-43AB-9B27-CA534218...,2014-02-08 10:01:36


In [12]:
sales_order_detail.set_role(["SalesOrderID", "SalesOrderDetailID", "ProductID", "SpecialOfferID"], getml.data.roles.join_key)
sales_order_detail.set_role(["OrderQty", "UnitPrice", "UnitPriceDiscount", "LineTotal"], getml.data.roles.numerical)
sales_order_detail.set_role("ModifiedDate", getml.data.roles.time_stamp)

sales_order_detail

name,ModifiedDate,SalesOrderID,SalesOrderDetailID,ProductID,SpecialOfferID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal,CarrierTrackingNumber,rowguid
role,time_stamp,join_key,join_key,join_key,join_key,numerical,numerical,numerical,numerical,unused_string,unused_string
unit,"time stamp, comparison only",,,,,,,,,,
0,2011-05-31,43659,1,776,1,1,2024.994,0,2024.994,4911-403C-98,B207C96D-D9E6-402B-8470-2CC176C4...
1,2011-05-31,43659,2,777,1,3,2024.994,0,6074.982,4911-403C-98,7ABB600D-1E77-41BE-9FE5-B9142CFC...
2,2011-05-31,43659,3,778,1,1,2024.994,0,2024.994,4911-403C-98,475CF8C6-49F6-486E-B0AD-AFC6A50C...
3,2011-05-31,43659,4,771,1,1,2039.994,0,2039.994,4911-403C-98,04C4DE91-5815-45D6-8670-F462719F...
4,2011-05-31,43659,5,772,1,1,2039.994,0,2039.994,4911-403C-98,5A74C7D2-E641-438E-A7AC-37BF2328...
,...,...,...,...,...,...,...,...,...,...,...
121312,2014-06-30,75122,121313,878,1,1,21.98,0,21.98,NULL,8CAD6675-18CC-4F47-8287-97B41A8E...
121313,2014-06-30,75122,121314,712,1,1,8.99,0,8.99,NULL,84F1C363-1C50-4442-BE16-541C59B6...


In [13]:
sales_order_reason.set_role("SalesOrderID", getml.data.roles.join_key)
sales_order_reason.set_role("SalesReasonID", getml.data.roles.categorical)

sales_order_reason

name,SalesOrderID,SalesReasonID,ModifiedDate
role,join_key,categorical,unused_string
0,43697,5,2011-05-31 00:00:00
1,43697,9,2011-05-31 00:00:00
2,43702,5,2011-06-01 00:00:00
3,43702,9,2011-06-01 00:00:00
4,43703,5,2011-06-01 00:00:00
,...,...,...
27642,75119,1,2014-06-30 00:00:00
27643,75120,1,2014-06-30 00:00:00
27644,75121,1,2014-06-30 00:00:00


In [14]:
special_offer.set_role(["SpecialOfferID"], getml.data.roles.join_key)
special_offer.set_role(["MinQty", "DiscountPct"], getml.data.roles.numerical)
special_offer.set_role(["Category", "Description", "Type"], getml.data.roles.categorical)
special_offer.set_role(["StartDate", "EndDate"], getml.data.roles.time_stamp)

special_offer

name,StartDate,EndDate,SpecialOfferID,Category,Description,Type,MinQty,DiscountPct,MaxQty,rowguid,ModifiedDate
role,time_stamp,time_stamp,join_key,categorical,categorical,categorical,numerical,numerical,unused_float,unused_string,unused_string
unit,"time stamp, comparison only","time stamp, comparison only",,,,,,,,,
0,2011-05-01,2014-11-30,1,No Discount,No Discount,No Discount,0,0,nan,0290C4F5-191F-4337-AB6B-0A2DDE03...,2011-04-01 00:00:00
1,2011-05-31,2014-05-30,2,Reseller,Volume Discount 11 to 14,Volume Discount,11,0.02,14,D7542EE7-15DB-4541-985C-5CC27AEF...,2011-05-01 00:00:00
2,2011-05-31,2014-05-30,3,Reseller,Volume Discount 15 to 24,Volume Discount,15,0.05,24,4BDBCC01-8CF7-40A9-B643-40EC5B71...,2011-05-01 00:00:00
3,2011-05-31,2014-05-30,4,Reseller,Volume Discount 25 to 40,Volume Discount,25,0.1,40,504B5E85-8F3F-4EBC-9E1D-C1BC5DEA...,2011-05-01 00:00:00
4,2011-05-31,2014-05-30,5,Reseller,Volume Discount 41 to 60,Volume Discount,41,0.15,60,677E1D9D-944F-4E81-90E8-47EB0A82...,2011-05-01 00:00:00
,...,...,...,...,...,...,...,...,...,...,...
11,2013-05-30,2013-07-14,12,Reseller,LL Road Frame Sale,Excess Inventory,0,0.35,nan,C0AF1C89-9722-4235-9248-3FBA4D9E...,2013-04-30 00:00:00
12,2013-05-30,2013-08-29,13,Reseller,Touring-3000 Promotion,New Product,0,0.15,nan,5061CCE4-E021-45A8-9A75-DFB36CBB...,2013-04-30 00:00:00


In [15]:
store.set_role(["SalesPersonID"], getml.data.roles.join_key)

store["test"] = store["ModifiedDate"].update(getml.data.random() > 0.5, "NULL")

store.set_role(["SalesPersonID"], getml.data.roles.join_key)
store.set_role(["test"], getml.data.roles.time_stamp)

store

name,test,SalesPersonID,BusinessEntityID,Name,Demographics,rowguid,ModifiedDate
role,time_stamp,join_key,unused_float,unused_string,unused_string,unused_string,unused_string
unit,"time stamp, comparison only",,,,,,
0,2014-09-12 11:15:07,279,292,Next-Door Bike Store,"<StoreSurvey xmlns=""http://schem...",A22517E3-848D-4EBE-B9D9-7437F343...,2014-09-12 11:15:07
1,NULL,276,294,Professional Sales and Service,"<StoreSurvey xmlns=""http://schem...",B50CA50B-C601-4A13-B07E-2C63862D...,2014-09-12 11:15:07
2,2014-09-12 11:15:07,277,296,Riders Company,"<StoreSurvey xmlns=""http://schem...",337C3688-1339-4E1A-A08A-B54B2356...,2014-09-12 11:15:07
3,NULL,275,298,The Bike Mechanics,"<StoreSurvey xmlns=""http://schem...",7894F278-F0C8-4D16-BD75-213FDBF1...,2014-09-12 11:15:07
4,NULL,286,300,Nationwide Supply,"<StoreSurvey xmlns=""http://schem...",C3FC9705-A8C4-4F3A-9550-EB2FA4B7...,2014-09-12 11:15:07
,...,...,...,...,...,...,...
696,NULL,282,1988,Retreat Inn,"<StoreSurvey xmlns=""http://schem...",EA21EC81-1BFA-4A07-9B4D-73D9852A...,2014-09-12 11:15:07
697,2014-09-12 11:15:07,281,1990,Technical Parts Manufacturing,"<StoreSurvey xmlns=""http://schem...",C8E3C4ED-8F58-4DB2-B600-E0CD11D9...,2014-09-12 11:15:07


In [16]:
sales_order_header["SalesPersonIDCat"] = sales_order_header["SalesPersonID"]
sales_order_header["TerritoryIDCat"] = sales_order_header["TerritoryID"]

sales_order_header.set_role(["CustomerID", "SalesOrderID", "SalesPersonID", "TerritoryID"], getml.data.roles.join_key)
sales_order_header.set_role(
    ["RevisionNumber", "OnlineOrderFlag", "SalesPersonIDCat", "TerritoryIDCat", "ShipMethodID"], 
    getml.data.roles.categorical)
sales_order_header.set_role(["SubTotal", "TaxAmt", "Freight", "TotalDue"], getml.data.roles.numerical)
sales_order_header.set_role(["OrderDate", "DueDate", "ShipDate", "ModifiedDate"], getml.data.roles.time_stamp)

sales_order_header

name,OrderDate,DueDate,ShipDate,ModifiedDate,CustomerID,SalesOrderID,SalesPersonID,TerritoryID,RevisionNumber,OnlineOrderFlag,ShipMethodID,SalesPersonIDCat,TerritoryIDCat,SubTotal,TaxAmt,Freight,TotalDue,Status,BillToAddressID,ShipToAddressID,CreditCardID,CurrencyRateID,churn,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CreditCardApprovalCode,Comment,rowguid
role,time_stamp,time_stamp,time_stamp,time_stamp,join_key,join_key,join_key,join_key,categorical,categorical,categorical,categorical,categorical,numerical,numerical,numerical,numerical,unused_float,unused_float,unused_float,unused_float,unused_float,unused_float,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string
unit,"time stamp, comparison only","time stamp, comparison only","time stamp, comparison only","time stamp, comparison only",,,,,,,,,,,,,,,,,,,,,,,,,
0,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29825,43659,279,5,8,0,5,279,5,20565.6206,1971.5149,616.0984,23153.2339,5,985,985,16281,nan,0,SO43659,PO522145787,10-4020-000676,105041Vi84182,NULL,79B65321-39CA-4115-9CBA-8FE0903E...
1,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29672,43660,279,5,8,0,5,279,5,1294.2529,124.2483,38.8276,1457.3288,5,921,921,5618,nan,0,SO43660,PO18850127500,10-4020-000117,115213Vi29411,NULL,738DC42D-D03B-48A1-9822-F95A67EA...
2,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29734,43661,282,6,8,0,5,282,6,32726.4786,3153.7696,985.553,36865.8012,5,517,517,1346,4,0,SO43661,PO18473189620,10-4020-000442,85274Vi6854,NULL,D91B9131-18A4-4A11-BC3A-90B6F53E...
3,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29994,43662,282,6,8,0,5,282,6,28832.5289,2775.1646,867.2389,32474.9324,5,482,482,10456,4,0,SO43662,PO18444174044,10-4020-000227,125295Vi53935,NULL,4A1ECFC0-CC3A-4740-B028-1C50BB48...
4,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29565,43663,276,4,8,0,5,276,4,419.4589,40.2681,12.5838,472.3108,5,1073,1073,4322,nan,0,SO43663,PO18009186470,10-4020-000510,45303Vi22691,NULL,9B1E7A40-6AE0-4AD3-811C-A6495185...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31460,2014-06-30,2014-07-12,2014-07-07,2014-07-07,11981,75119,nan,1,8,1,1,nan,1,42.28,3.3824,1.057,46.7194,5,17649,17649,6761,nan,nan,SO75119,NULL,10-4030-011981,429826Vi35166,NULL,9382F1C9-383A-435F-9449-0EECEA21...
31461,2014-06-30,2014-07-12,2014-07-07,2014-07-07,18749,75120,nan,6,8,1,1,nan,6,84.96,6.7968,2.124,93.8808,5,28374,28374,8925,nan,nan,SO75120,NULL,10-4030-018749,929849Vi46003,NULL,AE6A4FCF-FF73-4CD4-AF2C-5993D00D...


We must also define customer churn. In this case, we define customer churn as a customer not making another purchase within 180 days of his or her last purchase.

Thus, the churn variable is defined as follows:

- 0, if another purchase by the same customer has been made within 180 days after `OrderDate`
- 1, if no purchase by the same customer has been made within 180 days after `OrderDate`
- NULL, if `max(OrderDate) - OrderDate <= 180 days`

NULL targets can not be used in our analysis.

In [17]:
sales_order_header_pd = sales_order_header[["OrderDate", "CustomerID", "SalesOrderID"]].to_pandas()

repeat_purchases = sales_order_header_pd.merge(
    sales_order_header_pd[["OrderDate", "CustomerID"]],
    on="CustomerID",
    how="left",
)

repeat_purchases = repeat_purchases[
    repeat_purchases["OrderDate_y"] > repeat_purchases["OrderDate_x"]
]

repeat_purchases = repeat_purchases[
    repeat_purchases["OrderDate_y"] - repeat_purchases["OrderDate_x"] > pd.Timedelta('180 days')
]

repeat_purchases.groupby("SalesOrderID", as_index=False).aggregate({"CustomerID": "max"})

repeat_purchase_ids = {sid: True for sid in repeat_purchases["SalesOrderID"]}

cut_off_date = max(sales_order_header_pd["OrderDate"]) - pd.Timedelta('180 days')

churn = np.asarray([
    np.nan if order_date >= cut_off_date else 0 if order_id in repeat_purchase_ids else 1 
    for (order_date, order_id) in zip(sales_order_header_pd["OrderDate"], sales_order_header_pd["SalesOrderID"])
])

sales_order_header["churn"] = churn

sales_order_header = sales_order_header[~sales_order_header.churn.is_nan()].to_df("SalesOrderHeaderRefined")

sales_order_header.set_role("churn", getml.data.roles.target)

sales_order_header

name,OrderDate,DueDate,ShipDate,ModifiedDate,CustomerID,SalesOrderID,SalesPersonID,TerritoryID,churn,RevisionNumber,OnlineOrderFlag,ShipMethodID,SalesPersonIDCat,TerritoryIDCat,SubTotal,TaxAmt,Freight,TotalDue,Status,BillToAddressID,ShipToAddressID,CreditCardID,CurrencyRateID,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CreditCardApprovalCode,Comment,rowguid
role,time_stamp,time_stamp,time_stamp,time_stamp,join_key,join_key,join_key,join_key,target,categorical,categorical,categorical,categorical,categorical,numerical,numerical,numerical,numerical,unused_float,unused_float,unused_float,unused_float,unused_float,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string
unit,"time stamp, comparison only","time stamp, comparison only","time stamp, comparison only","time stamp, comparison only",,,,,,,,,,,,,,,,,,,,,,,,,
0,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29825,43659,279,5,0,8,0,5,279,5,20565.6206,1971.5149,616.0984,23153.2339,5,985,985,16281,nan,SO43659,PO522145787,10-4020-000676,105041Vi84182,NULL,79B65321-39CA-4115-9CBA-8FE0903E...
1,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29672,43660,279,5,0,8,0,5,279,5,1294.2529,124.2483,38.8276,1457.3288,5,921,921,5618,nan,SO43660,PO18850127500,10-4020-000117,115213Vi29411,NULL,738DC42D-D03B-48A1-9822-F95A67EA...
2,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29734,43661,282,6,0,8,0,5,282,6,32726.4786,3153.7696,985.553,36865.8012,5,517,517,1346,4,SO43661,PO18473189620,10-4020-000442,85274Vi6854,NULL,D91B9131-18A4-4A11-BC3A-90B6F53E...
3,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29994,43662,282,6,0,8,0,5,282,6,28832.5289,2775.1646,867.2389,32474.9324,5,482,482,10456,4,SO43662,PO18444174044,10-4020-000227,125295Vi53935,NULL,4A1ECFC0-CC3A-4740-B028-1C50BB48...
4,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29565,43663,276,4,0,8,0,5,276,4,419.4589,40.2681,12.5838,472.3108,5,1073,1073,4322,nan,SO43663,PO18009186470,10-4020-000510,45303Vi22691,NULL,9B1E7A40-6AE0-4AD3-811C-A6495185...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19699,2013-12-31,2014-01-12,2014-01-07,2014-01-07,20826,63358,nan,7,1,8,1,1,nan,7,1173.96,93.9168,29.349,1297.2258,5,24387,24387,3239,nan,SO63358,NULL,10-4030-020826,1142084Vi17039,NULL,41278FBB-3DD8-488B-AEA5-8BF4A6F1...
19700,2013-12-31,2014-01-12,2014-01-07,2014-01-07,24114,63359,nan,10,1,8,1,1,nan,10,1179.47,94.3576,29.4868,1303.3144,5,29682,29682,nan,10770,SO63359,NULL,10-4030-024114,NULL,NULL,749532D4-BFF7-4FEC-9F77-396C4A96...


In [18]:
split = getml.data.split.random(train=0.8, test=0.2)
split

,
0,train
1,train
2,train
3,test
4,train
,...


In [19]:
container = getml.data.Container(population=sales_order_header, split=split)

container.add(
    product=product,
    sales_order_detail=sales_order_detail, 
    sales_order_header=sales_order_header, 
    sales_order_reason=sales_order_reason,
    special_offer=special_offer,
    store=store,
)

container

,subset,name,rows,type
0,test,SalesOrderHeaderRefined,3879,View
1,train,SalesOrderHeaderRefined,15825,View
,alias,name,rows,type
0,product,Product,504,DataFrame
1,sales_order_detail,SalesOrderDetail,121317,DataFrame
2,sales_order_header,SalesOrderHeaderRefined,19704,DataFrame
3,sales_order_reason,SalesOrderHeaderSalesReason,27647,DataFrame
4,special_offer,SpecialOffer,16,DataFrame
5,store,Store,701,DataFrame


## 2. Predictive modeling

We loaded the data and defined the roles and units. Next, we create a getML pipeline for relational learning.

### 2.1 Define relational model

In [20]:
dm = getml.data.DataModel(sales_order_header.to_placeholder("population"))

dm.add(getml.data.to_placeholder(
    product=product,
    sales_order_detail=sales_order_detail, 
    sales_order_header=sales_order_header, 
    sales_order_reason=sales_order_reason,
    special_offer=special_offer,
    store=store,
))


dm.population.join(
    dm.sales_order_header,
    on="CustomerID",
    time_stamps="OrderDate",
    lagged_targets=True,
    horizon=getml.data.time.days(1),
)

dm.population.join(
    dm.sales_order_detail,
    on="SalesOrderID",
)

dm.population.join(
    dm.sales_order_reason,
    on="SalesOrderID", 
)

dm.population.join(
    dm.store,
    on="SalesPersonID", 
)

dm.sales_order_detail.join(
    dm.product,
    on="ProductID",
    relationship=getml.data.relationship.many_to_one,
)

dm.sales_order_detail.join(
    dm.special_offer,
    on="SpecialOfferID",
    relationship=getml.data.relationship.many_to_one,
)

dm

,data frames,staging table
0,population,POPULATION__STAGING_TABLE_1
1,"sales_order_detail, product, special_offer",SALES_ORDER_DETAIL__STAGING_TABLE_2
2,sales_order_header,SALES_ORDER_HEADER__STAGING_TABLE_3
3,sales_order_reason,SALES_ORDER_REASON__STAGING_TABLE_4
4,store,STORE__STAGING_TABLE_5


### 2.2 getML pipeline

<!-- #### 2.1.1  -->
__Set-up the feature learner & predictor__

In [21]:
seasonal = getml.preprocessors.Seasonal()

mapping = getml.preprocessors.Mapping()

fast_prop = getml.feature_learning.FastProp(
    loss_function=getml.feature_learning.loss_functions.CrossEntropyLoss,
    num_threads=1,    
    num_features=400,
)

relboost = getml.feature_learning.Relboost(
    loss_function=getml.feature_learning.loss_functions.CrossEntropyLoss,
    num_threads=1,    
)

predictor = getml.predictors.XGBoostClassifier(n_jobs=1)

__Build the pipeline__

In [22]:
pipe1 = getml.Pipeline(
    tags=['fast_prop'],
    data_model=dm,
    preprocessors=[mapping],
    feature_learners=[fast_prop],
    predictors=[predictor],
    include_categorical=True,
)

pipe1

Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=True,
         loss_function='CrossEntropyLoss',
         peripheral=['product', 'sales_order_detail', 'sales_order_header',
                     'sales_order_reason', 'special_offer', 'store'],
         predictors=['XGBoostClassifier'],
         preprocessors=['Mapping'],
         share_selected_features=0.5,
         tags=['fast_prop'])

In [23]:
pipe2 = getml.Pipeline(
    tags=['relboost'],
    data_model=dm,
    preprocessors=[seasonal, mapping],
    feature_learners=[relboost],
    predictors=[predictor],
    include_categorical=True,
)

pipe2

Pipeline(data_model='population',
         feature_learners=['Relboost'],
         feature_selectors=[],
         include_categorical=True,
         loss_function='CrossEntropyLoss',
         peripheral=['product', 'sales_order_detail', 'sales_order_header',
                     'sales_order_reason', 'special_offer', 'store'],
         predictors=['XGBoostClassifier'],
         preprocessors=['Seasonal', 'Mapping'],
         share_selected_features=0.5,
         tags=['relboost'])

### 2.3 Model training

In [24]:
pipe1.check(container.train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

The pipeline check generated 2 issues labeled INFO and 0 issues labeled WARNING.


,type,label,message
0,INFO,FOREIGN KEYS NOT FOUND,"When joining POPULATION__STAGING_TABLE_1 and SALES_ORDER_REASON__STAGING_TABLE_4 over 'SalesOrderID' and 'SalesOrderID', there are no corresponding entries for 33.769352% of entries in 'SalesOrderID' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys."
1,INFO,FOREIGN KEYS NOT FOUND,"When joining POPULATION__STAGING_TABLE_1 and STORE__STAGING_TABLE_5 over 'SalesPersonID' and 'SalesPersonID', there are no corresponding entries for 84.941548% of entries in 'SalesPersonID' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys."


In [25]:
pipe1.fit(container.train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

The pipeline check generated 2 issues labeled INFO and 0 issues labeled WARNING.
To see the issues in full, run .check() on the pipeline.

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
FastProp: Trying 710 features... 100% |██████████| [elapsed: 00:02, remaining: 00:00]          
FastProp: Building features... 100% |██████████| [elapsed: 00:01, remaining: 00:00]          
XGBoost: Training as predictor... 100% |██████████| [elapsed: 00:10, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:13.528577



Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=True,
         loss_function='CrossEntropyLoss',
         peripheral=['product', 'sales_order_detail', 'sales_order_header',
                     'sales_order_reason', 'special_offer', 'store'],
         predictors=['XGBoostClassifier'],
         preprocessors=['Mapping'],
         share_selected_features=0.5,
         tags=['fast_prop', 'container-EdpYd7'])

In [26]:
pipe2.check(container.train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:01, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

The pipeline check generated 2 issues labeled INFO and 0 issues labeled WARNING.


,type,label,message
0,INFO,FOREIGN KEYS NOT FOUND,"When joining POPULATION__STAGING_TABLE_1 and SALES_ORDER_REASON__STAGING_TABLE_4 over 'SalesOrderID' and 'SalesOrderID', there are no corresponding entries for 33.769352% of entries in 'SalesOrderID' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys."
1,INFO,FOREIGN KEYS NOT FOUND,"When joining POPULATION__STAGING_TABLE_1 and STORE__STAGING_TABLE_5 over 'SalesPersonID' and 'SalesPersonID', there are no corresponding entries for 84.941548% of entries in 'SalesPersonID' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys."


In [27]:
pipe2.fit(container.train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

The pipeline check generated 2 issues labeled INFO and 0 issues labeled WARNING.
To see the issues in full, run .check() on the pipeline.

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Relboost: Training features... 100% |██████████| [elapsed: 00:24, remaining: 00:00]          
Relboost: Building features... 100% |██████████| [elapsed: 00:03, remaining: 00:00]          
XGBoost: Training as predictor... 100% |██████████| [elapsed: 00:03, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:30.020059



Pipeline(data_model='population',
         feature_learners=['Relboost'],
         feature_selectors=[],
         include_categorical=True,
         loss_function='CrossEntropyLoss',
         peripheral=['product', 'sales_order_detail', 'sales_order_header',
                     'sales_order_reason', 'special_offer', 'store'],
         predictors=['XGBoostClassifier'],
         preprocessors=['Seasonal', 'Mapping'],
         share_selected_features=0.5,
         tags=['relboost', 'container-EdpYd7'])

### 2.4 Model evaluation

In [28]:
pipe1.score(container.test)

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
FastProp: Building features... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          



,date time,set used,target,accuracy,auc,cross entropy
0,2023-07-29 20:14:04,train,churn,0.9156,0.9742,0.2151
1,2023-07-29 20:14:36,test,churn,0.9139,0.9714,0.2238


In [29]:
pipe2.score(container.test)

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Relboost: Building features... 100% |██████████| [elapsed: 00:01, remaining: 00:00]          



,date time,set used,target,accuracy,auc,cross entropy
0,2023-07-29 20:14:35,train,churn,0.933,0.9836,0.1666
1,2023-07-29 20:14:37,test,churn,0.9288,0.9786,0.189


### 2.5 featuretools

In [30]:
population_train_pd = container.train.population.drop(container.train.population.roles.unused).to_pandas()
population_test_pd = container.test.population.drop(container.train.population.roles.unused).to_pandas()

In [31]:
product_pd = product.drop(product.roles.unused).to_pandas()
sales_order_detail_pd = sales_order_detail.drop(sales_order_detail.roles.unused).to_pandas()
sales_order_header_pd = sales_order_header.drop(sales_order_header.roles.unused).to_pandas()
sales_order_reason_pd = sales_order_reason.drop(sales_order_reason.roles.unused).to_pandas()
special_offer_pd = special_offer.drop(special_offer.roles.unused).to_pandas()
store_pd = store.drop(store.roles.unused).to_pandas()

featuretools does not support many-to-one joins. Therefore, we must manually merge `sales_order_detail_pd`, `product_pd` and `special_offer_pd`.

In [32]:
sales_order_detail_pd = sales_order_detail_pd.merge(
    product_pd,
    on="ProductID",
    how="left",
)

sales_order_detail_pd = sales_order_detail_pd.merge(
    special_offer_pd,
    on="SpecialOfferID",
    how="left",
)

del sales_order_detail_pd["SalesOrderDetailID"]
del sales_order_detail_pd["ProductID"]
del sales_order_detail_pd["SpecialOfferID"]

sales_order_detail_pd

,SalesOrderID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal,ModifiedDate,MakeFlag,ProductSubcategoryID,ProductModelID,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Category,Description,Type,MinQty,DiscountPct,StartDate,EndDate
0,43659,1.0,2024.994,0.0,2024.994,2011-05-31,1,1,19,100.0,75.0,1898.0944,3374.99,No Discount,No Discount,No Discount,0.0,0.0,2011-05-01,2014-11-30
1,43659,3.0,2024.994,0.0,6074.982,2011-05-31,1,1,19,100.0,75.0,1898.0944,3374.99,No Discount,No Discount,No Discount,0.0,0.0,2011-05-01,2014-11-30
2,43659,1.0,2024.994,0.0,2024.994,2011-05-31,1,1,19,100.0,75.0,1898.0944,3374.99,No Discount,No Discount,No Discount,0.0,0.0,2011-05-01,2014-11-30
3,43659,1.0,2039.994,0.0,2039.994,2011-05-31,1,1,19,100.0,75.0,1912.1544,3399.99,No Discount,No Discount,No Discount,0.0,0.0,2011-05-01,2014-11-30
4,43659,1.0,2039.994,0.0,2039.994,2011-05-31,1,1,19,100.0,75.0,1912.1544,3399.99,No Discount,No Discount,No Discount,0.0,0.0,2011-05-01,2014-11-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121312,75122,1.0,21.980,0.0,21.980,2014-06-30,0,30,121,4.0,3.0,8.2205,21.98,No Discount,No Discount,No Discount,0.0,0.0,2011-05-01,2014-11-30
121313,75122,1.0,8.990,0.0,8.990,2014-06-30,0,19,2,4.0,3.0,6.9223,8.99,No Discount,No Discount,No Discount,0.0,0.0,2011-05-01,2014-11-30
121314,75123,1.0,21.980,0.0,21.980,2014-06-30,0,30,121,4.0,3.0,8.2205,21.98,No Discount,No Discount,No Discount,0.0,0.0,2011-05-01,2014-11-30
121315,75123,1.0,159.000,0.0,159.000,2014-06-30,0,27,122,4.0,3.0,59.4660,159.00,No Discount,No Discount,No Discount,0.0,0.0,2011-05-01,2014-11-30


In [33]:
def prepare_sales_order_header(peripheral_pd, train_or_test):
    """
    Helper function that imitates the behavior of 
    the data model defined above.
    """
    peripheral_new = peripheral_pd.merge(
        train_or_test[["CustomerID", "OrderDate", "SalesOrderID"]],
        on="CustomerID"
    )

    peripheral_new = peripheral_new[
        peripheral_new["OrderDate_x"] < peripheral_new["OrderDate_y"]
    ]
    
    del peripheral_new["SalesOrderID_x"]
    del peripheral_new["OrderDate_y"]
    del peripheral_new["CustomerID"]
    del peripheral_new["SalesPersonIDCat"]
    del peripheral_new["TerritoryIDCat"]

    return peripheral_new.rename(columns={"OrderDate_x": "OrderDate", "SalesOrderID_y": "SalesOrderID"})

In [34]:
def prepare_store(peripheral_pd, train_or_test):
    """
    Helper function that imitates the behavior of 
    the data model defined above.
    """
    peripheral_new = peripheral_pd.merge(
        train_or_test[["SalesPersonID", "SalesOrderID"]],
        on="SalesPersonID"
    )
    
    return peripheral_new

In [35]:
store_train_pd = prepare_store(store_pd, population_train_pd)
store_test_pd = prepare_store(store_pd, population_test_pd)
store_train_pd

,SalesPersonID,test,SalesOrderID
0,279,2014-09-12 11:15:07,43659
1,279,2014-09-12 11:15:07,43660
2,279,2014-09-12 11:15:07,43681
3,279,2014-09-12 11:15:07,43685
4,279,2014-09-12 11:15:07,43695
...,...,...,...
142427,290,NaT,63216
142428,290,NaT,63217
142429,290,NaT,63223
142430,290,NaT,63282


In [36]:
sales_order_header_train_pd = prepare_sales_order_header(sales_order_header_pd, population_train_pd)
sales_order_header_test_pd = prepare_sales_order_header(sales_order_header_pd, population_test_pd)
sales_order_header_train_pd

,RevisionNumber,OnlineOrderFlag,ShipMethodID,SalesPersonID,TerritoryID,SubTotal,TaxAmt,Freight,TotalDue,churn,OrderDate,DueDate,ShipDate,ModifiedDate,SalesOrderID
1,8,0,5,279,5,20565.6206,1971.5149,616.0984,23153.2339,0.0,2011-05-31,2011-06-12,2011-06-07,2011-06-07,44305
2,8,0,5,279,5,20565.6206,1971.5149,616.0984,23153.2339,0.0,2011-05-31,2011-06-12,2011-06-07,2011-06-07,45061
3,8,0,5,279,5,20565.6206,1971.5149,616.0984,23153.2339,0.0,2011-05-31,2011-06-12,2011-06-07,2011-06-07,45779
4,8,0,5,279,5,20565.6206,1971.5149,616.0984,23153.2339,0.0,2011-05-31,2011-06-12,2011-06-07,2011-06-07,46604
5,8,0,5,279,5,20565.6206,1971.5149,616.0984,23153.2339,0.0,2011-05-31,2011-06-12,2011-06-07,2011-06-07,47693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39894,8,1,1,None,6,32.6000,2.6080,0.8150,36.0230,1.0,2013-12-15,2013-12-27,2013-12-22,2013-12-22,62799
39908,8,1,1,None,4,42.2800,3.3824,1.0570,46.7194,1.0,2013-12-13,2013-12-25,2013-12-20,2013-12-20,62770
39952,8,1,1,None,1,53.9900,4.3192,1.3498,59.6590,1.0,2013-12-15,2013-12-27,2013-12-22,2013-12-22,62796
40080,8,1,1,None,4,35.0000,2.8000,0.8750,38.6750,1.0,2013-12-19,2013-12-31,2013-12-26,2013-12-26,62643


In [37]:
del population_train_pd["CustomerID"]
del population_train_pd["SalesPersonIDCat"]

del population_test_pd["CustomerID"]
del population_test_pd["SalesPersonIDCat"]

featuretools's dataframes are similar for getML's container class.

In [38]:
dataframes_train = {
    "population" : (population_train_pd, "SalesOrderID"),
    "sales_order_header": (sales_order_header_train_pd, "index"),
    "sales_order_detail": (sales_order_detail_pd, "index"),
    "sales_order_reason": (sales_order_reason_pd, "index"),
    "store": (store_train_pd, "index"),
}

In [39]:
dataframes_test = {
    "population" : (population_test_pd, "SalesOrderID"),
    "sales_order_header": (sales_order_header_test_pd, "index"),
    "sales_order_detail": (sales_order_detail_pd, "index"),
    "sales_order_reason": (sales_order_reason_pd, "index"),
    "store": (store_test_pd, "index"),
}

featuretools's relationships are similar for getML's data model.

In [40]:
relationships = [
    ("population", "SalesOrderID", "sales_order_header", "SalesOrderID"),
    ("population", "SalesOrderID", "sales_order_detail", "SalesOrderID"),
    ("population", "SalesOrderID", "sales_order_reason", "SalesOrderID"),
    ("population", "SalesOrderID", "store", "SalesOrderID"),
]

In [41]:
featuretools_train_pd = featuretools.dfs(
    dataframes=dataframes_train,
    relationships=relationships,
    target_dataframe_name="population")[0]

/home/ubuntu/.local/lib/python3.10/site-packages/featuretools/entityset/entityset.py:1906: UserWarning: index index not found in dataframe, creating new integer column
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/featuretools/entityset/entityset.py:1906: UserWarning: index index not found in dataframe, creating new integer column
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/featuretools/entityset/entityset.py:1906: UserWarning: index index not found in dataframe, creating new integer column
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/featuretools/entityset/entityset.py:1906: UserWarning: index index not found in dataframe, creating new integer column
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/featuretools/entityset/entityset.py:414: UserWarning: Logical type PostalCode for child column SalesOrderID does not match parent column SalesOrderID logical type Integer. Changing child logical type to match pa

In [42]:
featuretools_test_pd = featuretools.dfs(
    dataframes=dataframes_test,
    relationships=relationships,
    target_dataframe_name="population")[0]

/home/ubuntu/.local/lib/python3.10/site-packages/featuretools/entityset/entityset.py:1906: UserWarning: index index not found in dataframe, creating new integer column
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/featuretools/entityset/entityset.py:754: UserWarning: A Woodwork-initialized DataFrame was provided, so the following parameters were ignored: index
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/featuretools/entityset/entityset.py:1906: UserWarning: index index not found in dataframe, creating new integer column
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/featuretools/entityset/entityset.py:414: UserWarning: Logical type PostalCode for child column SalesOrderID does not match parent column SalesOrderID logical type Integer. Changing child logical type to match parent.
  warnings.warn(


In [43]:
featuretools_train_pd

,RevisionNumber,OnlineOrderFlag,ShipMethodID,TerritoryIDCat,SalesPersonID,TerritoryID,SubTotal,TaxAmt,Freight,TotalDue,...,NUM_UNIQUE(sales_order_detail.YEAR(ModifiedDate)),NUM_UNIQUE(sales_order_detail.YEAR(StartDate)),MODE(store.DAY(test)),MODE(store.MONTH(test)),MODE(store.WEEKDAY(test)),MODE(store.YEAR(test)),NUM_UNIQUE(store.DAY(test)),NUM_UNIQUE(store.MONTH(test)),NUM_UNIQUE(store.WEEKDAY(test)),NUM_UNIQUE(store.YEAR(test))
SalesOrderID,,,,,,,,,,,,,,,,,,,,,
43659,8,0,5,5,279,5,20565.6206,1971.5149,616.0984,23153.2339,...,1,1,12,9,4,2014,1,1,1,1
43660,8,0,5,5,279,5,1294.2529,124.2483,38.8276,1457.3288,...,1,1,12,9,4,2014,1,1,1,1
43661,8,0,5,6,282,6,32726.4786,3153.7696,985.5530,36865.8012,...,1,1,12,9,4,2014,1,1,1,1
43663,8,0,5,4,276,4,419.4589,40.2681,12.5838,472.3108,...,1,1,12,9,4,2014,1,1,1,1
43664,8,0,5,1,280,1,24432.6088,2344.9921,732.8100,27510.4109,...,1,1,12,9,4,2014,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63358,8,1,1,7,<NA>,7,1173.9600,93.9168,29.3490,1297.2258,...,1,1,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>
63359,8,1,1,10,<NA>,10,1179.4700,94.3576,29.4868,1303.3144,...,1,1,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>
63360,8,1,1,7,<NA>,7,548.9800,43.9184,13.7245,606.6229,...,1,1,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>


In [44]:
featuretools_train = getml.data.DataFrame.from_pandas(featuretools_train_pd, "featuretools_train")
featuretools_test = getml.data.DataFrame.from_pandas(featuretools_test_pd, "featuretools_test")

In [45]:
featuretools_train.set_role("churn", getml.data.roles.target)
featuretools_train.set_role(featuretools_train.roles.unused_float, getml.data.roles.numerical)
featuretools_train.set_role(featuretools_train.roles.unused_string, getml.data.roles.categorical)

featuretools_train

name    churn   SalesPersonID   COUNT(sales_order_header)   COUNT(sales_order_detail)   ...   MEAN(store.SalesPersonID)
 role   target   categorical     categorical                 categorical                 ...                   numerical
    0        0   279             0                           12                          ...                        279 
    1        0   279             0                           2                           ...                        279 
    2        0   282             0                           15                          ...                        282 
    3        0   276             0                           1                           ...                        276 
    4        0   280             0                           8                           ...                        280 
           ...   ...             ...                         ...                                                     ...
15820        1   NULL            0                           4                           ...                        nan 
15821        1   NULL            0                           3                           ...                        nan 
15822        1   NULL            3                           2                           ...                        nan 
15823        1   NULL            1                           1                           ...                        nan 
15824        1   NULL            2                           2                           ...                        nan 

 name   MIN(store.SalesPersonID)   SKEW(store.SalesPersonID)   STD(store.SalesPersonID)   SUM(store.SalesPersonID)
 role                  numerical                   numerical                  numerical                  numerical
    0                       279                           0                          0                       22320
    1                       279                           0                          0                       22320
    2                       282                           0                          0                       20868
    3                       276                           0                          0                       10764
    4                       280                           0                          0                       10640
                             ...                         ...                        ...                        ...
15820                       nan                         nan                        nan                           0
15821                       nan                         nan                        nan                           0
15822                       nan                         nan                        nan                           0
15823                       nan                         nan                        nan                           0
15824                       nan                         nan                        nan                           0


15825 rows x 251 columns
memory usage: 26.02 MB
type: getml.DataFrame

In [46]:
featuretools_test.set_role("churn", getml.data.roles.target)
featuretools_test.set_role(featuretools_test.roles.unused_float, getml.data.roles.numerical)
featuretools_test.set_role(featuretools_test.roles.unused_string, getml.data.roles.categorical)

featuretools_test

name    churn   SalesPersonID   COUNT(sales_order_header)   COUNT(sales_order_detail)   ...   MEAN(store.SalesPersonID)
role   target   categorical     categorical                 categorical                 ...                   numerical
   0        0   282             0                           22                          ...                        282 
   1        0   283             0                           10                          ...                        283 
   2        0   283             0                           1                           ...                        283 
   3        0   279             0                           6                           ...                        279 
   4        0   277             0                           1                           ...                        277 
          ...   ...             ...                         ...                                                     ...
3874        1   NULL            1                           2                           ...                        nan 
3875        1   NULL            1                           4                           ...                        nan 
3876        1   NULL            1                           3                           ...                        nan 
3877        1   NULL            1                           4                           ...                        nan 
3878        1   NULL            0                           3                           ...                        nan 

name   MIN(store.SalesPersonID)   SKEW(store.SalesPersonID)   STD(store.SalesPersonID)   SUM(store.SalesPersonID)
role                  numerical                   numerical                  numerical                  numerical
   0                       282                           0                          0                       20868
   1                       283                           0                          0                       10754
   2                       283                           0                          0                       10754
   3                       279                           0                          0                       22320
   4                       277                           0                          0                       21052
                            ...                         ...                        ...                        ...
3874                       nan                         nan                        nan                           0
3875                       nan                         nan                        nan                           0
3876                       nan                         nan                        nan                           0
3877                       nan                         nan                        nan                           0
3878                       nan                         nan                        nan                           0


3879 rows x 251 columns
memory usage: 6.38 MB
type: getml.DataFrame

We train an untuned XGBoostRegressor on top of featuretools' features, just like we have done for getML's features.

Since some of featuretools features are categorical, we allow the pipeline to include these features as well. Other features contain NaN values, which is why we also apply getML's Imputation preprocessor.

In [47]:
imputation = getml.preprocessors.Imputation()

predictor = getml.predictors.XGBoostClassifier(n_jobs=1)

pipe3 = getml.Pipeline(
    tags=['featuretools'],
    preprocessors=[imputation],
    predictors=[predictor],
    include_categorical=True,
)

pipe3

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=True,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostClassifier'],
         preprocessors=['Imputation'],
         share_selected_features=0.5,
         tags=['featuretools'])

In [48]:
pipe3.fit(featuretools_train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

The pipeline check generated 0 issues labeled INFO and 20 issues labeled WARNING.
To see the issues in full, run .check() on the pipeline.

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
XGBoost: Training as predictor... 100% |██████████| [elapsed: 00:06, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:6.345451



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=True,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostClassifier'],
         preprocessors=['Imputation'],
         share_selected_features=0.5,
         tags=['featuretools'])

In [49]:
pipe3.score(featuretools_test)

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          



,date time,set used,target,accuracy,auc,cross entropy
0,2023-07-29 20:15:33,featuretools_train,churn,0.9113,0.9681,0.2371
1,2023-07-29 20:15:33,featuretools_test,churn,0.9095,0.9643,0.2467


### 2.6 Features

The most important features look as follows:

In [50]:
pipe1.features.to_sql()[pipe1.features.sort(by="importances")[0].name]

```sql
DROP TABLE IF EXISTS "FEATURE_1_304";

CREATE TABLE "FEATURE_1_304" AS
SELECT MIN( t1."orderdate" - t2."t4__startdate" ) AS "feature_1_304",
       t1.rowid AS rownum
FROM "POPULATION__STAGING_TABLE_1" t1
INNER JOIN "SALES_ORDER_DETAIL__STAGING_TABLE_2" t2
ON t1."salesorderid" = t2."salesorderid"
GROUP BY t1.rowid;
```

In [51]:
pipe2.features.to_sql()[pipe2.features.sort(by="importances")[0].name]

```sql
DROP TABLE IF EXISTS "FEATURE_1_9";

CREATE TABLE "FEATURE_1_9" AS
SELECT AVG( 
    CASE
        WHEN ( t1."orderdate" - t2."t4__startdate" > 66070588.235294 ) AND ( t1."salespersonidcat" IN ( '279', '282', '276', '280', '283', '277', '275', '278', '281', '289', '290', '287', '284', '286', '288', '285' ) ) AND ( t2."month__strftime__m__modifieddate__mapping_2_target_1_avg" > 0.763077 ) THEN 0.3796685623183497
        WHEN ( t1."orderdate" - t2."t4__startdate" > 66070588.235294 ) AND ( t1."salespersonidcat" IN ( '279', '282', '276', '280', '283', '277', '275', '278', '281', '289', '290', '287', '284', '286', '288', '285' ) ) AND ( t2."month__strftime__m__modifieddate__mapping_2_target_1_avg" <= 0.763077 OR t2."month__strftime__m__modifieddate__mapping_2_target_1_avg" IS NULL ) THEN -2.268454772087666
        WHEN ( t1."orderdate" - t2."t4__startdate" > 66070588.235294 ) AND ( t1."salespersonidcat" NOT IN ( '279', '282', '276', '280', '283', '277', '275', '278', '281', '289', '290', '287', '284', '286', '288', '285' ) OR t1."salespersonidcat" IS NULL ) AND ( t1."territoryidcat" IN ( '6' ) ) THEN -0.930969079655653
        WHEN ( t1."orderdate" - t2."t4__startdate" > 66070588.235294 ) AND ( t1."salespersonidcat" NOT IN ( '279', '282', '276', '280', '283', '277', '275', '278', '281', '289', '290', '287', '284', '286', '288', '285' ) OR t1."salespersonidcat" IS NULL ) AND ( t1."territoryidcat" NOT IN ( '6' ) OR t1."territoryidcat" IS NULL ) THEN 0.09675984351955905
        WHEN ( t1."orderdate" - t2."t4__startdate" <= 66070588.235294 OR t1."orderdate" IS NULL OR t2."t4__startdate" IS NULL ) AND ( t1."territoryidcat" IN ( '2', '10', '9', '8' ) ) AND ( t2."t3__productsubcategoryid__mapping_2_target_1_avg" > 0.552951 ) THEN 1.011399875662692
        WHEN ( t1."orderdate" - t2."t4__startdate" <= 66070588.235294 OR t1."orderdate" IS NULL OR t2."t4__startdate" IS NULL ) AND ( t1."territoryidcat" IN ( '2', '10', '9', '8' ) ) AND ( t2."t3__productsubcategoryid__mapping_2_target_1_avg" <= 0.552951 OR t2."t3__productsubcategoryid__mapping_2_target_1_avg" IS NULL ) THEN -2.509642008381071
        WHEN ( t1."orderdate" - t2."t4__startdate" <= 66070588.235294 OR t1."orderdate" IS NULL OR t2."t4__startdate" IS NULL ) AND ( t1."territoryidcat" NOT IN ( '2', '10', '9', '8' ) OR t1."territoryidcat" IS NULL ) AND ( t2."t3__productmodelid" IN ( '1', '14', '19', '21', '2', '10', '11', '4', '17', '35', '34', '36', '33', '24', '28', '37', '26', '29', '30', '32', '9', '6', '13', '8', '7', '104', '106', '59', '61', '52', '56', '42', '45', '46', '50', '51', '123', '124', '125', '78', '116', '38', '111', '118', '107', '127', '128', '79', '80', '81', '84', '66', '67', '62', '63', '64', '68', '69', '70', '53', '103', '47', '48', '102', '99', '101', '98', '95', '97' ) ) THEN -2.174803681005045
        WHEN ( t1."orderdate" - t2."t4__startdate" <= 66070588.235294 OR t1."orderdate" IS NULL OR t2."t4__startdate" IS NULL ) AND ( t1."territoryidcat" NOT IN ( '2', '10', '9', '8' ) OR t1."territoryidcat" IS NULL ) AND ( t2."t3__productmodelid" NOT IN ( '1', '14', '19', '21', '2', '10', '11', '4', '17', '35', '34', '36', '33', '24', '28', '37', '26', '29', '30', '32', '9', '6', '13', '8', '7', '104', '106', '59', '61', '52', '56', '42', '45', '46', '50', '51', '123', '124', '125', '78', '116', '38', '111', '118', '107', '127', '128', '79', '80', '81', '84', '66', '67', '62', '63', '64', '68', '69', '70', '53', '103', '47', '48', '102', '99', '101', '98', '95', '97' ) OR t2."t3__productmodelid" IS NULL ) THEN -1.519452639124237
        ELSE NULL
    END
) AS "feature_1_9",
       t1.rowid AS rownum
FROM "POPULATION__STAGING_TABLE_1" t1
INNER JOIN "SALES_ORDER_DETAIL__STAGING_TABLE_2" t2
ON t1."salesorderid" = t2."salesorderid"
GROUP BY t1.rowid;
```

### 2.7 Productionization

It is possible to productionize the pipeline by transpiling the features into production-ready SQL code. Please also refer to getML's `sqlite3` and `spark` modules.


In [54]:
# Creates a folder containing the SQL code.
pipe2.features.to_sql().save("adventure_works", remove=True)

In [55]:
# Creates a folder containing the SQL code.
pipe2.features.to_sql(dialect=getml.pipeline.dialect.spark_sql).save("adventure_works_spark", remove=True)

### 2.8 Discussion

For a more convenient overview, we summarize our results into a table.

Name                 | Accuracy   | AUC    | Cross entropy
-------------------- | ---------- | -------| -------------
getML: FastProp      |     91.26% | 0.9711 | 0.2217
getML: Relboost      |     92.73% | 0.9776 | 0.1929
featuretools         |     91.00% | 0.9677 | 0.2374

The picture we get is very consistent: Relboost outperforms FastProp and FastProp outperforms featuretools for all three measures.

These results also clearly demonstrate that this is a synthetic dataset - in the real-world, customer churn can not be predicted at this level of accuracy.

## 3. Conclusion

We demonstrated how getML can be used for a classic customer churn project. We have also benchmarked against featuretools. We found that getML outperforms featuretools.

## References

Motl, Jan, and Oliver Schulte. "The CTU prague relational learning repository." arXiv preprint arXiv:1511.03086 (2015).

# Next Steps

This tutorial benchmarked getML against academic state-of-the-art algorithms from relational learning literature and getML's qualities with respect to categorical data.

If you are interested in further real-world applications of getML, head back to the [notebook overview](welcome.md) and choose one of the remaining examples.

Here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Feature learning with Multirel](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#multirel)
* [Feature learning with Relboost](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relboost)

# Get in contact

If you have any question schedule a [call with Alex](https://go.getml.com/meetings/alexander-uhlig/getml-demo), the co-founder of getML, or write us an [email](team@getml.com). Prefer a private demo of getML? Just contact us to make an appointment.